In [1]:
%load_ext autoreload
%autoreload 2

In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
import math
import copy
import spacy
import pathlib
from pathlib import Path


from funnel_transformer_conan import *
from data_loader import *

from torch.utils.tensorboard import SummaryWriter
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/funnel_experiment2')
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)


cuda:1


In [71]:
data_en = EnFrDataset(used_abridged_data=True, max_seq_length=100)

Reading the dataframe and storing untokenized pairs...


100%|██████████| 5000/5000 [00:00<00:00, 163147.90it/s]


Adding sentences to Langs amd geting data pairs...


100%|██████████| 5000/5000 [00:00<00:00, 6349.99it/s]


Creating tokenized pairs of english and french sentences...


100%|██████████| 5000/5000 [00:00<00:00, 12229.87it/s]


In [72]:
train_dataloader = DataLoader(data_en, batch_size=32, shuffle=False, num_workers=6)

In [75]:
from data_loader import *
from funnel_transformer_conan import *
# full_dataset_path = Path('data/en-fr.csv')
# abridged_dataset_path = Path('data/en-fr-abridged.csv')

# ds = pd.read_csv(abridged_dataset_path)
# english, french, pairs = read_lang('en', 'fr', abridged_dataset)

# src_vocab_size = 5000
# tgt_vocab_size = 5000
# d_model = 512
# num_heads = 8
# num_layers = 6
# d_ff = 2048
# max_seq_length = 100
# dropout = 0.1

src_vocab_size = data_en.get_src_lang_size()
tgt_vocab_size = data_en.get_tgt_lang_size()
d_model = 512
num_heads = 4
d_ff = 2014
max_seq_length = 100
encoder_blocks = 3
print(100%(2**encoder_blocks)!=0)
print(100/(2**encoder_blocks)==0)
assert(100%(2**(encoder_blocks-1))!=0 or 100/(2**(encoder_blocks-1))==0)==0,"dafaq"
decoder_blocks = 6
dropout = 0.1

transformer = Funnel_Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, decoder_blocks, encoder_blocks, d_ff, max_seq_length, dropout)
# # Generate random sample data
# src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
# tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)



loss_criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
batch_size = 64
Funnel_Transformer = Funnel_Transformer.to(device)
total_loss = 0.0
for epoch in range(1000):
    epoch_loss = []
    transformer.train()
    for it in train_dataloader:
        src_data, tgt_data = it
        src_data = src_data.to(device)
        tgt_data = tgt_data.to(device)
        optimizer.zero_grad()
        output= transformer(src_data, tgt_data[:, :-1])
        break
        loss = loss_criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    epoch_loss.append(loss.item())
    writer.add_scalar('training loss',
                    total_loss,
                        epoch)
    if epoch % 5 == 0:
        print(f"Epoch: {epoch+1}, Epoch Loss: {total_loss}")
    total_loss = 0.0
    

True
False


AssertionError: dafaq

In [15]:
a = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout),nn.MaxPool1d(3,stride=2) for _ in range(num_layers)])

SyntaxError: invalid syntax (139525130.py, line 1)

In [3]:
print(torch.randint(1,10,(64,100)))


tensor([[9, 5, 4,  ..., 6, 7, 4],
        [8, 9, 3,  ..., 6, 8, 8],
        [8, 3, 2,  ..., 6, 7, 6],
        ...,
        [6, 9, 6,  ..., 8, 2, 6],
        [6, 7, 9,  ..., 8, 1, 4],
        [3, 9, 2,  ..., 4, 1, 3]])
